In [1]:
import numpy as np
import tensorflow as tf
import time
# import random

2022-01-02 19:04:37.976605: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
is_cuda_gpu_min_3

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-01-02 19:04:41.264304: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


True

2022-01-02 19:04:41.265607: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-02 19:04:41.294114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-02 19:04:41.294257: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2022-01-02 19:04:41.294272: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-02 19:04:41.295640: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-02 19:04:41.295666: I tensorflow/stream_executor/pl

In [3]:
train = np.load('../data/freebase15k_numpy/train.npy')
validation = np.load('../data/freebase15k_numpy/validation.npy')
test = np.load('../data/freebase15k_numpy/test.npy')

entities = np.load('../data/freebase15k_numpy/entities.npy')
relations = np.load('../data/freebase15k_numpy/relations.npy')

train = tf.convert_to_tensor(train.astype(dtype=np.int16), dtype=tf.int32)
validation = tf.convert_to_tensor(validation.astype(dtype=np.int16), dtype=tf.int32)
test = tf.convert_to_tensor(test.astype(dtype=np.int16), dtype=tf.int16)
relations = tf.convert_to_tensor(relations.astype(dtype=np.int16), dtype=tf.int16)
entities = tf.convert_to_tensor(entities.astype(dtype=np.float64), dtype=tf.float64)

2022-01-02 19:04:44.201782: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-02 19:04:44.201925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-02 19:04:44.202190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2022-01-02 19:04:44.202247: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-02 19:04:44.202275: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-02 19:04:44.202286: I tensorflow/stream_executor/platfor

In [4]:
iteration = tf.constant(500)
dim = tf.constant(100)
kns = tf.constant(5)
alpha = tf.Variable(0.000001, dtype = tf.float64)
beta = tf.Variable(0.0000001, dtype = tf.float64)
z = tf.constant(0.5, dtype = tf.float64)
nn1 = tf.random.uniform( shape=(entities.shape[0], dim),
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn1')
      
nn0 =tf.random.uniform(shape=(entities.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64)), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn0')
nn2 = tf.random.uniform(shape=(relations.shape[0], dim), 
      minval=tf.math.truediv(z , tf.cast(dim,tf.float64)), 
      maxval=tf.math.truediv(z , tf.cast(dim,tf.float64)),
      dtype=tf.dtypes.float64, seed=None, name='nn2')

In [8]:
def calculateRank(triple):   
    selectedEntities = tf.gather(entities, 0, axis=1)

    _head_index = tf.gather(triple,0)
    _tail_index = tf.gather(triple,1)
    _relation_index = tf.gather(triple,2)  
    # print(tf.where(tf.math.logical_not( tf.math.logical_or( (tf.gather(train, 1, axis=1) == _tail_index) , (tf.gather(train, 2, axis=1) == _relation_index)))))
    # selectedEntities = tf.where(np.isin(selectedEntities , train[tf.where((tf.gather(train, 1, axis=1) ==_tail_index) & (tf.gather(train, 2, axis=1) ==_relation_index))][0], invert =True ))[0]

    if(tf.shape((tf.where((tf.gather(train, 1, axis=1) == _tail_index) & (tf.gather(train, 2, axis=1) == _relation_index))))[0]>0):
        selectedEntities = tf.where(np.isin(selectedEntities , train[tf.where((tf.gather(train, 1, axis=1) ==_tail_index) & (tf.gather(train, 2, axis=1) ==_relation_index))][0], invert =True ))[0]



    if(len((tf.where((test[:,1] == _tail_index) & (test[:,2] == _relation_index))[0]))> 0):
        selectedEntities = tf.where(np.isin(selectedEntities , test[tf.where((test[:,1] ==_tail_index) & (test[:,2] ==_relation_index))][0], invert =True ))[0]
    if(len((tf.where((validation[:,1] == _tail_index) & (validation[:,2] == _relation_index))[0]))> 0):
        selectedEntities = tf.where(np.isin(selectedEntities , validation[tf.where((validation[:,1] ==_tail_index) & (validation[:,2] ==_relation_index))][0], invert =True ))[0]
    couraptedH = np.empty((selectedEntities.shape[0],3))
    couraptedH[:,0] =selectedEntities
    couraptedH[:,1] =_tail_index
    couraptedH[:,2] =_relation_index
    
    selectedEntities = entities[:,0]
    if(len((tf.where((train[:,0] == _head_index) & (tf.gather(train, 2, axis=1) == _relation_index))[0]))> 0):
        selectedEntities = tf.where(np.isin(selectedEntities , train[tf.where((train[:,0] == _head_index) & (tf.gather(train, 2, axis=1) == _relation_index))][0], invert =True ))[0]
    if(len((tf.where((test[:,0] == _head_index) & (test[:,2] == _relation_index))[0]))> 0):
        selectedEntities = tf.where(np.isin(selectedEntities , test[tf.where((test[:,0] ==_head_index) & (test[:,2] ==_relation_index))][0], invert =True ))[0]
    if(len((tf.where((validation[:,0] == _head_index) & (validation[:,2] == _relation_index))[0]))> 0):
        selectedEntities = tf.where(np.isin(selectedEntities , validation[tf.where((validation[:,0] ==_head_index) & (validation[:,2] ==_relation_index))][0], invert =True ))[0]
    couraptedT = np.empty((selectedEntities.shape[0],3))
    couraptedT[:,0] = _head_index
    couraptedT[:,1] = selectedEntities
    couraptedT[:,2] =_relation_index
    courapted = np.concatenate((couraptedH, couraptedT), axis=0)
    courapted = courapted.astype(np.int16)
    courapted = np.vstack([courapted, triple])
    
    p =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(nn1[courapted[:,1]],tf.transpose(nn0[courapted[:,0]] + nn2[_relation_index] ),axes=1 )))
    
    courapted =np.concatenate((courapted, p[:, None]), axis=1)
    courapted = courapted[np.argsort(courapted[:, -3])]
    return tf.where((courapted[:,0] == _head_index) & (courapted[:,1] == _tail_index))[0][0]

In [ ]:
def mrr(ranks):
    inverse = []
    for rank in ranks:
        inverse.append(float(1)/rank)
    summ = sum(inverse)
    return float(1)/len(inverse) * summ

# 483142    

In [ ]:
start = time.time()
paddings = tf.constant([[0, 1,], [0, 0]])
t_row = train.shape[0]
for x in range(0 , iteration):
    # tf.random.shuffle(train, seed=None, name=None)
    with tf.device('/cpu:0'):
        total_samples  = tf.gather( tf.random.categorical([tf.gather(entities, 2, axis=1)], tf.math.multiply(kns,t_row) , dtype=None, seed=None),0)
    for tIndex in range(0 , t_row):
        triple = tf.gather(train, tIndex)
        _head_index = tf.gather(triple,0)
        _relation_index = tf.gather(triple,2)
        _vHead = tf.gather(nn0,_head_index)
        _vRel = tf.gather(nn2,_relation_index)

        samples = [tf.slice(total_samples,begin=[tf.math.multiply(tIndex, kns)],size=[kns])]
        samples = tf.pad(samples, paddings, constant_values=0)
        samples =  tf.cast(samples, tf.int64)         
        samples = tf.transpose(tf.concat([samples, [[tf.gather(triple,1)],[1]]], 1))
        indices = tf.gather(samples, 0, axis=1)
        _nn1_samples = tf.gather(nn1, indices)
        _sigmoid =tf.math.sigmoid(tf.tensordot( _nn1_samples , tf.transpose(tf.math.add(_vHead , _vRel)) , axes=1))
        cost = tf.math.subtract(tf.cast(tf.gather(samples, 1, axis=1), tf.float64) , _sigmoid)
        g = tf.math.multiply(alpha , cost)
        g1 = tf.math.multiply(beta , cost)
        _nn1_samples = tf.math.add(tf.math.multiply(tf.gather(nn1, indices) , tf.reshape(g, (6, 1))) , _nn1_samples)
        nn1 = tf.tensor_scatter_nd_update(nn1,tf.expand_dims(indices, 1),_nn1_samples)

        _nn2_sample = tf.math.add_n([tf.math.reduce_sum(tf.math.multiply(_vRel , tf.reshape(g1, (g1.shape[0], 1))), axis=0, keepdims=False)  , _vRel, tf.math.reduce_sum(_nn1_samples, axis=0, keepdims=False)])
        indices = tf.constant(_relation_index)
        nn2 = tf.tensor_scatter_nd_update(nn2,tf.expand_dims([indices], 1), tf.reshape(_nn2_sample,(1, _nn2_sample.shape[0])))
  
        indices = tf.constant(_head_index)
        _nn0_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(_vHead , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _nn2_sample)
        nn0 = tf.tensor_scatter_nd_update(nn0,tf.expand_dims([indices], 1), tf.reshape(_nn0_sample,(1, _nn0_sample.shape[0])))
    print(time.time() - start)        
    # if x % 10 == 0 and x > 20:
# _mrrList = []
# # for triple in validation:
# _mrrList.append(calculateRank(validation[0,:]))
# print('total one sample calc time =>  ' , time.time() - start) 
# 1696.807552099228

In [10]:
start = time.time()
_mrrList = []
triple = tf.gather(validation, 0)
_mrrList.append(calculateRank(triple))
print('total one sample calc time =>  ' , time.time() - start) 
print(train.shape)

tf.Tensor(
[[     0]
 [     1]
 [     2]
 ...
 [483139]
 [483140]
 [483141]], shape=(475829, 1), dtype=int64)
total one sample calc time =>   0.01224207878112793
(483142, 3)


In [36]:
import collections

In [105]:
t = tf.constant([ [1,2,5],[1,2,6], [3,3,7], [2, 5, 4],[0,3,3]])
e = tf.constant([0,1,2,3,4,5,6,7])
trip = tf.constant([1,2,2])
indexes = tf.where(tf.math.logical_and( (tf.gather(t, 0, axis=1) == 1) , (tf.gather(t, 1, axis=1) == 2)))
indexes = tf.reshape(indexes, (indexes.shape[0] ))
existEntites = tf.gather(tf.gather(t, indexes), 2, axis=1)
# existEntites = tf.reshape(existEntites ,  (existEntites.shape[0] , 1) )
print(e)
print(existEntites)



scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), existEntites, shape)
scatter = tf.subtract(e , scatter)
print(tf.where(tf.math.greater(scatter, 0)) )




tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tf.Tensor([5 6], shape=(2,), dtype=int32)
tf.Tensor(
[[1]
 [2]
 [3]
 [4]
 [7]], shape=(5, 1), dtype=int64)
